# Creating hybrid dataset

In this file, a hybrid dataset is created from the byte file and ASM file. Byte file features are extracted using two models, one is a simple CNN and another one is CNN with pretrained encoder layers. All byte file data pass through both of these pretrained models for feature extraction and then concatenated with the ASM file feature which was selected using SVM classifier 

Importing libraries and initailizing variables

In [1]:
import torch
import torch.utils.data as data
import numpy as np
import os
from cnn_v1 import ConvNet_single
import torch.nn.functional as F
from dataloader_csv import CustomDatasetFromImages
import csv
from cnn_stacking import autoencoder,autoencoder1
from cnn_stacking import Cnn_Stacking

As pretrained models are store in Cuda version, so for loading them GPU must be available 

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
if torch.cuda.is_available(): 
    print("gpu available")

Loading byte File data

In [4]:
test_path=  'data_to_test.csv'

test_data = CustomDatasetFromImages(test_path)

test_data_loader  = data.DataLoader(test_data, batch_size=1, shuffle=False) 

train_path=  'data_to_tra.csv'

train_data = CustomDatasetFromImages(train_path)

train_data_loader  = data.DataLoader(train_data, batch_size=1, shuffle=False) 

valid_path=  'data_to_val.csv'

valid_data = CustomDatasetFromImages(valid_path)

valid_data_loader  = data.DataLoader(valid_data, batch_size=1, shuffle=False) 

fulL_train_path=  'data_to_traFull.csv'

fulL_train_data = CustomDatasetFromImages(fulL_train_path)

fulL_train_data_loader  = data.DataLoader(fulL_train_data, batch_size=1, shuffle=False) 


Creating the First model, which will extract features from byte files

In [5]:
model= ConvNet_single().to(device)

Loading the best model state during training

In [6]:
path_to_checkpoint = '..../saved_model_state/'

scores=[]
List= os.listdir(path_to_checkpoint)
for i in range(len(List)):
    scores.append(float(List[i].split()[0]))
max_idx= np.argmin(scores)
min_loss_path= path_to_checkpoint+List[max_idx]
if (len(List)!=0):        
    checkpoint= torch.load(min_loss_path)
    model.load_state_dict(checkpoint['state_dict'])
    model.eval()
    print("model initialized")

Loading the ASM file data(opcode frequencies)

In [7]:
path_tr='n_500_data_to_tra.csv'
path_futr='n_500_data_to_traFull.csv'
path_te='n_500_data_to_test.csv'
path_val='n_500_data_to_val.csv'

train = np.genfromtxt(path_tr, delimiter=",", dtype=str)

val = np.genfromtxt(path_val, delimiter=",", dtype=str)

test = np.genfromtxt(path_te, delimiter=",", dtype=str)

full_train = np.genfromtxt(path_futr, delimiter=",", dtype=str)

train=train[1:,:]

val=val[1:,:]

test=test[1:,:]

full_train=full_train[1:,:]


Following piece of code extracts the feature from test data of byte file and concatenate them to top 116 features selected by SVM

In [8]:
data_to_write_test=[]

for image,name,label in test_data_loader:
    image=image.to(device)

    output=model(image)
    output=F.softmax(output)
    output = output.reshape(output.size(0), -1)
    
    output=output.to('cpu')
    output=output.detach().numpy()
    output=output.tolist()
    output=output[0]
    for i in test:
        if i[-2]==name[0]:
            print(i[-2],name[0])
            for j in i[0:116]:
                output.append(j)
            output.append(i[-2])
            output.append(i[-1])
            
    data_to_write_test.append(output)

myFile = open('final_hybrid_test.csv', 'w', newline='')  

with myFile:  
    writer = csv.writer(myFile)
    writer.writerows(data_to_write_test)


Following piece of code extracts the feature from train data of byte file and concatenate them to top 116 features selected by SVM

In [9]:
data_to_write_train=[]

for image,name,label in train_data_loader:
    image=image.to(device)
    output=model(image)
    output=F.softmax(output) 
    output = output.reshape(output.size(0), -1)
    
    output=output.to('cpu')
    output=output.detach().numpy()
    output=output.tolist()
    output=output[0]
    for i in train:
        if i[-2]==name[0]:
            print(i[-2],name[0])
            for j in i[0:116]:
                output.append(j)
            output.append(i[-2])
            output.append(i[-1])
            
    data_to_write_train.append(output)

myFile = open('final_hybrid_train.csv', 'w', newline='')  

with myFile:  
    writer = csv.writer(myFile)
    writer.writerows(data_to_write_train)


Following piece of code extracts the feature from validation data of byte file and concatenate them to top 116 features selected by SVM

In [10]:
data_to_write_val=[]

for image,name,label in valid_data_loader:
    image=image.to(device)
    output=model(image)
    output=F.softmax(output)
    
    output = output.reshape(output.size(0), -1)
    
    output=output.to('cpu')
    output=output.detach().numpy()
    output=output.tolist()
    output=output[0]
    for i in val:
        if i[-2]==name[0]:
            print(i[-2],name[0])
            for j in i[0:116]:
                output.append(j)
            output.append(i[-2])
            output.append(i[-1])
            
    data_to_write_val.append(output)
myFile = open('final_hybrid_valid.csv', 'w', newline='')  

with myFile:  
    writer = csv.writer(myFile)
    writer.writerows(data_to_write_val)

Following piece of code extracts the feature from overall train data of byte file and concatenate them to top 116 features selected by SVM 

In [11]:
data_to_write_full_train=[]

for image,name,label in fulL_train_data_loader:
    image=image.to(device)
    output=model(image)
    output=F.softmax(output)
    
    output = output.reshape(output.size(0), -1)
    
    output=output.to('cpu')
    output=output.detach().numpy()
    output=output.tolist()
    output=output[0]
    for i in full_train:
        if i[-2]==name[0]:
            print(i[-2],name[0])
            for j in i[0:116]:
                output.append(j)
            output.append(i[-2])
            output.append(i[-1])
            
    data_to_write_full_train.append(output)

myFile = open('final_hybrid_full_train.csv', 'w', newline='')  

with myFile:  
    writer = csv.writer(myFile)
    writer.writerows(data_to_write_full_train)

Now create the second model (CNN with pretrained encoder layers) from which feature of byte file will be extracted

In [12]:
auto=autoencoder().to(device).float()
auto1=autoencoder1().to(device).float()

model=Cnn_Stacking(auto,auto1).to(device).float()


Loading the best model state during FINE TUNING

In [13]:
path_to_checkpoint='...//cnn_auto_saved_model//'

scores=[]
List= os.listdir(path_to_checkpoint)
for i in range(len(List)):
    scores.append(float(List[i].split()[0]))
max_idx= np.argmin(scores)
min_loss_path= path_to_checkpoint+List[max_idx]
if (len(List)!=0):        
    checkpoint= torch.load(min_loss_path)
    model.load_state_dict(checkpoint['state_dict'])
    model.eval()
    print("model initialized")

Loading data from above processed csv ( extracted features(byte file) + 116 features(asm) ) 

In [14]:
path_tr='final_hybrid_train.csv'

path_futr='final_hybrid_full_train.csv'

path_te='final_hybrid_test.csv'

path_val='final_hybrid_valid.csv'

train = np.genfromtxt(path_tr, delimiter=",", dtype=str)

val = np.genfromtxt(path_val, delimiter=",", dtype=str)

test = np.genfromtxt(path_te, delimiter=",", dtype=str)

full_train = np.genfromtxt(path_futr, delimiter=",", dtype=str)

Following piece of code extracts the feature from test data of byte file using fine tunned CNN and concatenate them to previous extracted and selected features to generate hybrid data

In [15]:
data_to_write_test=[]

for image,name,label in test_data_loader:
    image=image.to(device)
    output=model(image)
    output=F.softmax(output)
    output = output.reshape(output.size(0), -1)
    output=output.to('cpu')
    output=output.detach().numpy()
    output=output.tolist()
    output=output[0]
    for i in test:
        if i[-2]==name[0]:
            print(i[-2],name[0])
            for j in i:
                output.append(j)
            
    data_to_write_test.append(output)

myFile = open('final_hybrid_test.csv', 'w', newline='')  

with myFile:  
    writer = csv.writer(myFile)
    writer.writerows(data_to_write_test)


Following piece of code extracts the feature from train data of byte file using fine tunned CNN and concatenate them to previous extracted and selected features to generate hybrid data

In [16]:
data_to_write_train=[]

for image,name,label in train_data_loader:
    image=image.to(device)
    output=model(image)
    output=F.softmax(output)
    
    output = output.reshape(output.size(0), -1)
    
    output=output.to('cpu')
    output=output.detach().numpy()
    output=output.tolist()
    output=output[0]
    for i in train:
        if i[-2]==name[0]:
            print(i[-2],name[0])
            for j in i:
                output.append(j)
            
    data_to_write_train.append(output)

myFile = open('final_hybrid_train.csv', 'w', newline='')  

with myFile:  
    writer = csv.writer(myFile)
    writer.writerows(data_to_write_train)

Following piece of code extracts the feature from the validation data of byte file using fine tunned CNN and concatenate them to previous extracted and selected features to generate hybrid data 

In [17]:
data_to_write_val=[]

for image,name,label in valid_data_loader:
    image=image.to(device)
    output=model(image)
    output=F.softmax(output)
    
    output = output.reshape(output.size(0), -1)
    
    output=output.to('cpu')
    output=output.detach().numpy()
    output=output.tolist()
    output=output[0]
    for i in val:
        if i[-2]==name[0]:
            print(i[-2],name[0])
            for j in i:
                output.append(j)
            
    data_to_write_val.append(output)
myFile = open('final_hybrid_valid.csv', 'w', newline='')  

with myFile:  
    writer = csv.writer(myFile)
    writer.writerows(data_to_write_val)

Following piece of code extracts the feature from overall data of byte file using fine tunned CNN and concatenate them to previous extracted and selected features to generate hybrid data 

In [18]:
data_to_write_full_train=[]

for image,name,label in fulL_train_data_loader:
    image=image.to(device)
    output=model(image)
    output=F.softmax(output)
    
    output = output.reshape(output.size(0), -1)
    
    output=output.to('cpu')
    output=output.detach().numpy()
    output=output.tolist()
    output=output[0]
    for i in full_train:
        if i[-2]==name[0]:
            print(i[-2],name[0])
            for j in i:
                output.append(j)

    
    data_to_write_full_train.append(output)
    
myFile = open('final_hybrid_full_train.csv', 'w', newline='')  

with myFile:  
    writer = csv.writer(myFile)
    writer.writerows(data_to_write_full_train)